In [84]:
#open data
include("2_4.jl")
inputfilepath = "data_2-4"
inputfilename = "Data_test_5_2_3.txt"
nb_dem, nb_prod, nb_mag, nb_noeuds, matrice_S, matrice_Q, matrice_R = read_data_24(inputfilepath, inputfilename)

(5, 3, 2, 7, Array{Float64,N} where N[[9.0, 8.0, 6.0], [10.0, 6.0, 7.0]], Array{Float64,N} where N[[3.0, 1.0, 3.0], [4.0, 4.0, 1.0], [3.0, 1.0, 3.0], [3.0, 3.0, 1.0], [4.0, 3.0, 3.0]], Array{Float64,N} where N[[0.0, 570.0, 630.0, 503.0, 113.0, 438.0, 654.0], [570.0, 0.0, 68.0, 225.0, 460.0, 178.0, 401.0], [630.0, 68.0, 0.0, 285.0, 523.0, 246.0, 381.0], [503.0, 225.0, 285.0, 0.0, 392.0, 100.0, 600.0], [113.0, 460.0, 523.0, 392.0, 0.0, 325.0, 590.0], [438.0, 178.0, 246.0, 100.0, 325.0, 0.0, 512.0], [654.0, 401.0, 381.0, 600.0, 590.0, 512.0, 0.0]])

In [85]:
using Cbc
using JuMP
using LinearAlgebra

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, x[1:nb_noeuds, 1:nb_noeuds, 1:nb_mag] >= 0, Int)
@variable(model, A[1:nb_mag, 1:nb_prod, 1:nb_dem] >= 0, Int)

# define objective function
# x[i, j, k] = 1 si le livreur k traverse le chemin v(i, j) | 0 sinon
@objective(model, Min, sum(sum(sum(x[i, j, k] * matrice_R[i][j] for i in 1:nb_noeuds) for j in 1:nb_noeuds) for k in 1:nb_mag))

#Constraints

# le meme point ne peut pas etre son propre predecesseur / successeur 
for i=1:nb_noeuds
    for k=1:nb_mag
        @constraint(model, x[i, i, k] == 0)
    end
end

# chaque point i possede un seul successeur
for i=1:nb_noeuds
    for k=1:nb_mag
       @constraint(model, sum(x[i, 1:nb_noeuds, k]) == 1)
    end
end

# chaque point j possede un seul predecesseur
for j=1:nb_noeuds
    for k=1:nb_mag
        @constraint(model, sum(x[1:nb_noeuds, j, k]) == 1)
    end
end

# pas d'aller retour
for i=1:nb_noeuds
    for j=1:nb_noeuds
        for k=1:nb_mag
            @constraint(model, x[i, j, k] + x[j, i, k] <= 1)
        end
    end
end

# pas de transitions entre les magasins
for i in 1:nb_mag
    for j in 1:nb_mag
        for k in 1:nb_mag
            @constraint(model, x[i, j, k] == 0)
        end
    end 
end

# Aucun magasin ne peut livrer plus que ce qu'il a en stock
for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model,  sum(A[i,j,k] for k in 1:nb_dem) <= matrice_S[i][j])
    end
end

# Le nombre total demandé par chaque client
for j in 1:nb_prod
    for k in 1:nb_dem
        @constraint(model,  sum(A[i,j,k] for i in 1:nb_mag ) == matrice_Q[k][j])
    end
end

# un livreur ne peut pas servir autre que ses clients
for k in 1:nb_dem
    for i in 1:nb_mag
        @constraint(model, sum(A[i, j, k] for j in 1:nb_prod) >= sum(x[k + nb_mag, l, i] for l in 1:nb_noeuds ))
        @constraint(model, sum(A[i, j, k] for j in 1:nb_prod) >= sum(x[l, k + nb_mag, i] for l in 1:nb_noeuds ))
    end
end

optimize!(model)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3732 - 0.00 seconds
Cgl0004I processed model has 94 rows, 95 columns (95 integer (84 of which binary)) and 450 elements
Cbc0012I Integer solution of 3732 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 3732, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 3732 to 3732
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were a

In [86]:
for i in 1:nb_noeuds
    for j in 1:nb_noeuds
        print("$(value(x[:, j, 1][i]))   ")
    end
    print("\n")
end

0.0   0.0   0.0   0.0   1.0   0.0   0.0   
0.0   0.0   1.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   0.0   1.0   
0.0   1.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   0.0   1.0   0.0   
0.0   0.0   0.0   1.0   0.0   0.0   0.0   
1.0   0.0   0.0   0.0   0.0   0.0   0.0   


In [90]:
for k in 1:nb_mag
    for i in 1:nb_noeuds
         for j in 1:nb_noeuds
           if (value(x[i,j,k]) != 0)
                println("\t le livreur $k traverse $i vers $(j)")
            end 
        end
    end 
end
println("----------------------------------------------------------")
for k in 1:nb_dem
    for j in 1:nb_prod
        for i in 1:nb_mag
            if (value(A[i,j,k])) != 0
                println("\t le client $k prend $(value(A[i,j,k])) du P$j a partir du magasin $i")
            end
        end
    end
end

	 le livreur 1 traverse 1 vers 5
	 le livreur 1 traverse 2 vers 3
	 le livreur 1 traverse 3 vers 7
	 le livreur 1 traverse 4 vers 2
	 le livreur 1 traverse 5 vers 6
	 le livreur 1 traverse 6 vers 4
	 le livreur 1 traverse 7 vers 1
	 le livreur 2 traverse 1 vers 5
	 le livreur 2 traverse 2 vers 3
	 le livreur 2 traverse 3 vers 7
	 le livreur 2 traverse 4 vers 2
	 le livreur 2 traverse 5 vers 6
	 le livreur 2 traverse 6 vers 4
	 le livreur 2 traverse 7 vers 1
----------------------------------------------------------
	 le client 1 prend 1.0 du P1 a partir du magasin 1
	 le client 1 prend 2.0 du P1 a partir du magasin 2
	 le client 1 prend 1.0 du P2 a partir du magasin 2
	 le client 1 prend 3.0 du P3 a partir du magasin 2
	 le client 2 prend 4.0 du P1 a partir du magasin 2
	 le client 2 prend 4.0 du P2 a partir du magasin 1
	 le client 2 prend 1.0 du P3 a partir du magasin 2
	 le client 3 prend 2.0 du P1 a partir du magasin 1
	 le client 3 prend 1.0 du P1 a partir du magasin 2
	 le client